In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON

In [2]:
method_algo = 'lpcc'
model_algo = 'rnn'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [3]:
def create_dense_model():
    # build the network architecture
    model = keras.Sequential([
        # 1st hidden layer
        keras.layers.SimpleRNN(512, input_shape=[inputs.shape[1], inputs.shape[2]]),

        keras.layers.Dropout(0.3),

        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])

    return model

In [4]:
# SPLIT DOS DADOS
random_state = 42
# for random_state in [5438, 53, 14]:
#     for _ in range(4):
# split data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

inputs_train, inputs_valid, targets_train, targets_valid = train_test_split(inputs_train,
                                                                            targets_train,
                                                                            test_size=0.2,
                                                                            stratify=targets_train,
                                                                            random_state=random_state)


In [5]:
# SALVA UMA CÓPIA DOS DADOS DE TESTE

# data = {
#     "mapping": mapping,
#     "labels": targets_test,
#     "mfcc": inputs_test,
# }

# JSON.create_json_file(f'datatest/deep/datatest_{random_state}_{inputs.shape[0]}.json', data)

In [6]:
# CRIA O MODELO
model = create_dense_model()

# COMPILA A REDE
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [ ]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
                    validation_data=(inputs_valid, targets_valid),
                    epochs=10000,
                    batch_size=128,
                    callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000
89/89 [==============================] - 11s 109ms/step - loss: 5.4978 - accuracy: 0.0103 - val_loss: 4.7639 - val_accuracy: 0.0106
Epoch 2/10000
89/89 [==============================] - 8s 93ms/step - loss: 5.0735 - accuracy: 0.0117 - val_loss: 4.7355 - val_accuracy: 0.0134
Epoch 3/10000
89/89 [==============================] - 10s 107ms/step - loss: 5.0444 - accuracy: 0.0095 - val_loss: 4.7201 - val_accuracy: 0.0134
Epoch 4/10000
89/89 [==============================] - 10s 110ms/step - loss: 5.0158 - accuracy: 0.0113 - val_loss: 4.7052 - val_accuracy: 0.0106
Epoch 5/10000
89/89 [==============================] - 8s 93ms/step - loss: 5.0281 - accuracy: 0.0106 - val_loss: 4.6940 - val_accuracy: 0.0120
Epoch 6/10000
89/89 [==============================] - 8s 92ms/step - loss: 5.0095 - accuracy: 0.0127 - val_loss: 4.6986 - val_accuracy: 0.0163
Epoch 7/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.9938 - accuracy: 0.0105 - val_loss: 4.6989 - val_acc

89/89 [==============================] - 9s 96ms/step - loss: 4.8784 - accuracy: 0.0120 - val_loss: 4.6726 - val_accuracy: 0.0113
Epoch 62/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.8821 - accuracy: 0.0121 - val_loss: 4.6740 - val_accuracy: 0.0134
Epoch 63/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.8947 - accuracy: 0.0111 - val_loss: 4.6718 - val_accuracy: 0.0138
Epoch 64/10000
89/89 [==============================] - 9s 98ms/step - loss: 4.8895 - accuracy: 0.0107 - val_loss: 4.6716 - val_accuracy: 0.0170
Epoch 65/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.8731 - accuracy: 0.0123 - val_loss: 4.6673 - val_accuracy: 0.0138
Epoch 66/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.8722 - accuracy: 0.0138 - val_loss: 4.6629 - val_accuracy: 0.0131
Epoch 67/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.8591 - accuracy: 0.0143 - val_loss: 4.6651 - val_accuracy: 0.0127


89/89 [==============================] - 8s 94ms/step - loss: 4.7469 - accuracy: 0.0141 - val_loss: 4.6228 - val_accuracy: 0.0202
Epoch 118/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.7650 - accuracy: 0.0158 - val_loss: 4.6230 - val_accuracy: 0.0212
Epoch 119/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.7632 - accuracy: 0.0128 - val_loss: 4.6267 - val_accuracy: 0.0141
Epoch 120/10000
89/89 [==============================] - 8s 95ms/step - loss: 4.7464 - accuracy: 0.0146 - val_loss: 4.6217 - val_accuracy: 0.0209
Epoch 121/10000
89/89 [==============================] - 10s 107ms/step - loss: 4.7417 - accuracy: 0.0163 - val_loss: 4.6285 - val_accuracy: 0.0198
Epoch 122/10000
89/89 [==============================] - 11s 119ms/step - loss: 4.7547 - accuracy: 0.0134 - val_loss: 4.6232 - val_accuracy: 0.0219
Epoch 123/10000
89/89 [==============================] - 9s 100ms/step - loss: 4.7483 - accuracy: 0.0139 - val_loss: 4.6196 - val_accura

89/89 [==============================] - 10s 111ms/step - loss: 4.6622 - accuracy: 0.0145 - val_loss: 4.5719 - val_accuracy: 0.0191
Epoch 174/10000
89/89 [==============================] - 9s 102ms/step - loss: 4.6589 - accuracy: 0.0177 - val_loss: 4.5688 - val_accuracy: 0.0195
Epoch 175/10000
89/89 [==============================] - 9s 106ms/step - loss: 4.6637 - accuracy: 0.0191 - val_loss: 4.5714 - val_accuracy: 0.0216
Epoch 176/10000
89/89 [==============================] - 9s 106ms/step - loss: 4.6360 - accuracy: 0.0204 - val_loss: 4.5634 - val_accuracy: 0.0230
Epoch 177/10000
89/89 [==============================] - 9s 102ms/step - loss: 4.6496 - accuracy: 0.0185 - val_loss: 4.5634 - val_accuracy: 0.0202
Epoch 178/10000
89/89 [==============================] - 9s 100ms/step - loss: 4.6380 - accuracy: 0.0207 - val_loss: 4.5589 - val_accuracy: 0.0233
Epoch 179/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.6360 - accuracy: 0.0224 - val_loss: 4.5648 - val_accu

Epoch 229/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.5326 - accuracy: 0.0249 - val_loss: 4.4825 - val_accuracy: 0.0223
Epoch 230/10000
89/89 [==============================] - 9s 99ms/step - loss: 4.5353 - accuracy: 0.0230 - val_loss: 4.4798 - val_accuracy: 0.0272
Epoch 231/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.5063 - accuracy: 0.0252 - val_loss: 4.4839 - val_accuracy: 0.0262
Epoch 232/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.5383 - accuracy: 0.0221 - val_loss: 4.4775 - val_accuracy: 0.0248
Epoch 233/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.5390 - accuracy: 0.0223 - val_loss: 4.4749 - val_accuracy: 0.0255
Epoch 234/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.5198 - accuracy: 0.0244 - val_loss: 4.4715 - val_accuracy: 0.0258
Epoch 235/10000
89/89 [==============================] - 8s 95ms/step - loss: 4.5079 - accuracy: 0.0228 - val_loss: 4.4841 -

...

Epoch 901/10000
89/89 [==============================] - 10s 110ms/step - loss: 4.0557 - accuracy: 0.0428 - val_loss: 4.1154 - val_accuracy: 0.0407
Epoch 902/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.0749 - accuracy: 0.0456 - val_loss: 4.1062 - val_accuracy: 0.0417
Epoch 903/10000
89/89 [==============================] - 9s 106ms/step - loss: 4.0524 - accuracy: 0.0495 - val_loss: 4.1004 - val_accuracy: 0.0386
Epoch 904/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.0785 - accuracy: 0.0416 - val_loss: 4.1129 - val_accuracy: 0.0414
Epoch 905/10000
89/89 [==============================] - 9s 97ms/step - loss: 4.0723 - accuracy: 0.0440 - val_loss: 4.0982 - val_accuracy: 0.0414
Epoch 906/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0528 - accuracy: 0.0527 - val_loss: 4.0983 - val_accuracy: 0.0382
Epoch 907/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0355 - accuracy: 0.0518 - val_loss: 4.108

Epoch 957/10000
89/89 [==============================] - 10s 111ms/step - loss: 4.0344 - accuracy: 0.0487 - val_loss: 4.0970 - val_accuracy: 0.0435
Epoch 958/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0481 - accuracy: 0.0508 - val_loss: 4.0862 - val_accuracy: 0.0449
Epoch 959/10000
89/89 [==============================] - 9s 100ms/step - loss: 4.0453 - accuracy: 0.0451 - val_loss: 4.0840 - val_accuracy: 0.0432
Epoch 960/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0257 - accuracy: 0.0488 - val_loss: 4.1260 - val_accuracy: 0.0435
Epoch 961/10000
89/89 [==============================] - 11s 126ms/step - loss: 4.0284 - accuracy: 0.0465 - val_loss: 4.0905 - val_accuracy: 0.0421
Epoch 962/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.0188 - accuracy: 0.0583 - val_loss: 4.0879 - val_accuracy: 0.0449
Epoch 963/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0375 - accuracy: 0.0460 - val_loss: 4.0

Epoch 1013/10000
89/89 [==============================] - 9s 105ms/step - loss: 4.0256 - accuracy: 0.0529 - val_loss: 4.0768 - val_accuracy: 0.0432
Epoch 1014/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.0350 - accuracy: 0.0507 - val_loss: 4.0794 - val_accuracy: 0.0446
Epoch 1015/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0048 - accuracy: 0.0571 - val_loss: 4.0780 - val_accuracy: 0.0364
Epoch 1016/10000
89/89 [==============================] - 10s 109ms/step - loss: 4.0148 - accuracy: 0.0503 - val_loss: 4.1070 - val_accuracy: 0.0428
Epoch 1017/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0299 - accuracy: 0.0471 - val_loss: 4.0866 - val_accuracy: 0.0396
Epoch 1018/10000
89/89 [==============================] - 10s 112ms/step - loss: 4.0250 - accuracy: 0.0474 - val_loss: 4.0732 - val_accuracy: 0.0432
Epoch 1019/10000
89/89 [==============================] - 9s 105ms/step - loss: 4.0201 - accuracy: 0.0481 - val_l

89/89 [==============================] - 9s 104ms/step - loss: 3.9951 - accuracy: 0.0465 - val_loss: 4.0662 - val_accuracy: 0.0428
Epoch 1069/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0144 - accuracy: 0.0515 - val_loss: 4.0601 - val_accuracy: 0.0453
Epoch 1070/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.0155 - accuracy: 0.0479 - val_loss: 4.0939 - val_accuracy: 0.0400
Epoch 1071/10000
89/89 [==============================] - 10s 114ms/step - loss: 4.0137 - accuracy: 0.0497 - val_loss: 4.0663 - val_accuracy: 0.0488
Epoch 1072/10000
89/89 [==============================] - 9s 101ms/step - loss: 4.0079 - accuracy: 0.0526 - val_loss: 4.0626 - val_accuracy: 0.0439
Epoch 1073/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.0025 - accuracy: 0.0516 - val_loss: 4.0552 - val_accuracy: 0.0467
Epoch 1074/10000
89/89 [==============================] - 8s 93ms/step - loss: 3.9928 - accuracy: 0.0582 - val_loss: 4.0727 - val_a

Epoch 1124/10000
89/89 [==============================] - 8s 93ms/step - loss: 3.9883 - accuracy: 0.0511 - val_loss: 4.0381 - val_accuracy: 0.0485
Epoch 1125/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0412 - accuracy: 0.0502 - val_loss: 4.0425 - val_accuracy: 0.0474
Epoch 1126/10000
89/89 [==============================] - 9s 104ms/step - loss: 4.0156 - accuracy: 0.0518 - val_loss: 4.0748 - val_accuracy: 0.0474
Epoch 1127/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0140 - accuracy: 0.0523 - val_loss: 4.0810 - val_accuracy: 0.0460
Epoch 1128/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0116 - accuracy: 0.0554 - val_loss: 4.0564 - val_accuracy: 0.0488
Epoch 1129/10000
89/89 [==============================] - 11s 119ms/step - loss: 3.9924 - accuracy: 0.0578 - val_loss: 4.0623 - val_accuracy: 0.0474
Epoch 1130/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0066 - accuracy: 0.0552 - val_loss

In [ ]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [ ]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

In [ ]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                           f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')